https://huggingface.co/blog/how-to-generate

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

/mnt/m2-mint/NLP-Zad2/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "sdadas/polish-gpt2-medium"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [3]:
tokenizer.vocab_size

51200

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=False)
)

In [5]:
dataset = pd.read_csv("data/annotations_all_batches - SENTENCE.csv")
dataset

,text,Olek,Kuba,Zgodne?,Stachu,Finalna anotacja
0,"""Generalnie nie mam żadnych zastrzeżeń i jak n...",2,2,T,NaN,2
1,Etui na pierwszy rzut oka wydaje się bardzo ła...,0,0,T,NaN,0
2,"Statyw fajnie wykonany, wysoki i ku mojemu zdz...",1,1,T,NaN,1
3,"przepraszam ze tak dlugo nie odpowiadalem ,ale...",2,2,T,NaN,2
4,"Brak opakowania, instrukcji przez co występują...",0,1,N,0.0,0
...,...,...,...,...,...,...
133,Szkło zle przylega do telefonu. Tylko cieniutk...,0,0,T,NaN,0
134,Sprzedającego ten produkt omijać z daleka. Spr...,0,0,T,NaN,0
135,Bardzo kiepskie wykonanie nie polecam zakupu. ...,0,0,T,NaN,0
136,"""Do codziennego użytkowania są rewelacyjne. Od...",2,2,T,NaN,2


In [6]:
import textwrap

sample_sentence = dataset["text"].iloc[10]
print(textwrap.fill(sample_sentence, width=100))

Dobry i tani oczyszczacz. Ma wskaźnik jakości powietrza, który przetestowałam i sprawdził się bardzo
dobrze. Przed otwarciem okna w pokoju stan powietrza był dobry, ale po otwarciu ( a tego wieczory
był w powietrzu wyraźny smog) już po minucie oczyszczacz przeskoczył na zły  stan powietrza i
włączył  intensywne oczyszczanie.


In [7]:
def replace_initial_sentence_with_ellipsis(initial_sentence, generated_sentence):
    last_n_characters = (
        initial_sentence[-10:] if len(initial_sentence) > 10 else initial_sentence
    )
    generated_sentence = generated_sentence.replace(initial_sentence, "")
    return f"...{last_n_characters}{generated_sentence}"

In [8]:
def decode_generated_text(sample_sentence, generated, with_ellipsis=True):
    generated_text = tokenizer.batch_decode(generated, skip_special_tokens=True)
    if with_ellipsis:
        return [
            replace_initial_sentence_with_ellipsis(sample_sentence, sentence)
            for sentence in generated_text
        ]
    return generated_text

In [9]:
inputs = tokenizer(sample_sentence, return_tensors="pt").to(device)
generated = model.generate(
    **inputs,
    max_new_tokens=60,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_text = decode_generated_text(sample_sentence, generated)
print(textwrap.fill(generated_text[0], width=100))

...yszczanie. A po paru minutach znowu zaczął się psuć. Na szczęście filtr HEPA nie jest aż tak
bardzo wrażliwy na kurz. Włączając go nie trzeba pamiętać o wymianie filtrów, bo oczyszczacz się sam
wyłącza. Na szczęście przy starcie oczyszczacz sam włącza się i wyłącza (w trybie awaryjnym) i


# Greedy search and beam search test

In [10]:
# greedy search test - we can see that it always returns the same sentence

# What's more, it returns very repetitive and boring sentences - it
# omits high probability words hidden behind low probability ones

# It also doesn't generate any new sentences between runs, it just repeats the same one

for _ in range(5):
    generated = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_text = decode_generated_text(sample_sentence, generated)
    print(textwrap.fill(generated_text[0], width=100))
    print("")

...yszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po otwarciu okna w pokoju stan
powietrza był już bardzo zły. Po otwarciu okna w pokoju stan powietrza był już bardzo zły, ale po
otwarciu okna w pokoju stan powietrza był już bardzo zły. Po otwarciu okna w pokoju stan powietrza
był już bardzo zły,

...yszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po otwarciu okna w pokoju stan
powietrza był już bardzo zły. Po otwarciu okna w pokoju stan powietrza był już bardzo zły, ale po
otwarciu okna w pokoju stan powietrza był już bardzo zły. Po otwarciu okna w pokoju stan powietrza
był już bardzo zły,

...yszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po otwarciu okna w pokoju stan
powietrza był już bardzo zły. Po otwarciu okna w pokoju stan powietrza był już bardzo zły, ale po
otwarciu okna w pokoju stan powietrza był już bardzo zły. Po otwarciu okna w pokoju stan powietrza
był już bardzo zły,

...yszczanie. Po otwarciu okna w pokoju sta

In [11]:
# beam search test

# still very repetitive and boring sentences

# It also doesn't generate any new sentences between runs, it just repeats the same one

for _ in range(5):
    generated = model.generate(
        **inputs,
        max_new_tokens=60,
        num_beams=10,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_text = decode_generated_text(sample_sentence, generated)
    print(textwrap.fill(generated_text[0], width=100))
    print()

...yszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po otwarciu ( a tego wieczory
był w powietrzu wyraźny smog) już po minucie oczyszczacz przeskoczył na zły  stan powietrza i
włączył  intensywne oczyszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po
otwarciu ( a tego wieczory

...yszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po otwarciu ( a tego wieczory
był w powietrzu wyraźny smog) już po minucie oczyszczacz przeskoczył na zły  stan powietrza i
włączył  intensywne oczyszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po
otwarciu ( a tego wieczory

...yszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po otwarciu ( a tego wieczory
był w powietrzu wyraźny smog) już po minucie oczyszczacz przeskoczył na zły  stan powietrza i
włączył  intensywne oczyszczanie. Po otwarciu okna w pokoju stan powietrza był dobry, ale po
otwarciu ( a tego wieczory

...yszczanie. Po otwarciu okna w pokoju stan powietrza 

In [12]:
# penalty for repetition (n_gram)

# Generated sentences are not repetitive, but they start diverging from the original context
# (online product reviews) and become less coherent

# It also doesn't generate any new sentences between runs, it just repeats the same one

for _ in range(5):
    generated = model.generate(
        **inputs,
        max_new_tokens=60,
        num_beams=10,
        early_stopping=True,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_text = decode_generated_text(sample_sentence, generated)
    print(textwrap.fill(generated_text[0], width=100))
    print()

...yszczanie. Okazało się, że poziom PM2,5 jest w normie, a poziom pyłów PM10 nie przekracza normy.
W związku z tym mam pytanie do Pana Ministra: Czy jest szansa na to, aby w najbliższym czasie
zakupić oczyszczacze powietrza? Z poważaniem Poseł Maria Zbyrowska Dębica, dnia

...yszczanie. Okazało się, że poziom PM2,5 jest w normie, a poziom pyłów PM10 nie przekracza normy.
W związku z tym mam pytanie do Pana Ministra: Czy jest szansa na to, aby w najbliższym czasie
zakupić oczyszczacze powietrza? Z poważaniem Poseł Maria Zbyrowska Dębica, dnia

...yszczanie. Okazało się, że poziom PM2,5 jest w normie, a poziom pyłów PM10 nie przekracza normy.
W związku z tym mam pytanie do Pana Ministra: Czy jest szansa na to, aby w najbliższym czasie
zakupić oczyszczacze powietrza? Z poważaniem Poseł Maria Zbyrowska Dębica, dnia

...yszczanie. Okazało się, że poziom PM2,5 jest w normie, a poziom pyłów PM10 nie przekracza normy.
W związku z tym mam pytanie do Pana Ministra: Czy jest szansa na to, aby w 

# Sampling tests

In [13]:
# no top-K sampling

from transformers import set_seed

set_seed(11)

# the text seems not to be coherent and human-like


generated = model.generate(
    **inputs,
    max_new_tokens=60,
    top_k=0,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = decode_generated_text(sample_sentence, generated)
print(textwrap.fill(generated_text[0], width=100))

...yszczanie. Panel sterowania urządzenie posiada LED. Oświetlenie czytelne, ale nie oślepiające.
Czujnik ruchu. Tutaj nie był potrzebny mi czujnik ruchu. Ale ja nie lubię, jak światło zapala się i
gaśnie. Ogólnie oceniam ten oczyszczacz jako dobry i tani oczyszczacz i na pewno rewelacyjny


In [14]:
# lowering temperature in no top-k sampling

set_seed(11)

# the text seems to be more coherent and more human-like, thanks to the lower temperature
# (lower temperature means less randomness in the generated text)
# (as temperature approaches 0.0, the model becomes deterministic)


generated = model.generate(
    **inputs,
    max_new_tokens=60,
    top_k=0,
    do_sample=True,
    temperature=0.4,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = decode_generated_text(sample_sentence, generated)
print(textwrap.fill(generated_text[0], width=100))
print()

...yszczanie. Panel sterowania jest bardzo czytelny, a obsługa intuicyjna. Na początku może wydawać
się, że oczyszczacz jest głośny, ale po kilku minutach użytkowania jest cicho i spokojnie. Polecam.



In [15]:
# introducing top-k sampling
# it means that top-k most probable tokens are selected and the rest is discarded.
# After selection, the probability mass of these tokens is redistributed between them

# the text seems to be very human-like

# issues with top-k
# - when the next-token distribution is very sharp and points to a single token (or n tokens, n < k), top-k sampling
#   will select very implausible tokens anyway to match the k number

set_seed(11)

generated = model.generate(
    **inputs,
    max_new_tokens=60,
    top_k=50,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = decode_generated_text(sample_sentence, generated)
print(textwrap.fill(generated_text[0], width=100))
print()

...yszczanie. Panel sterowania posiada funkcję zmiany poziomu mocy urządzenia. Na wyświetlaczu
wyraźnie widać poziom wilgotności - ok. 3-4 proc. W aplikacji znajduje się wiele dodatkowych
funkcji, także takie jak wyświetlanie informacji na temat aktualnego zanieczyszczenia powietrza oraz
jego klasy oraz liczby wykonanych pomiarów. Urządzenie spełnia swoje zadanie rewelacyjnie



In [16]:
# introducing top-p sampling
# unlike in top-k, top-p dynamically selects the tokens so that the sum of their probability mass
# exceeds p. Again it redistributes the probability mass between the selected tokens.

# it also produces human-like results

set_seed(11)

generated = model.generate(
    **inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.90,
    top_k=0,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = decode_generated_text(sample_sentence, generated)
print(textwrap.fill(generated_text[0], width=100))
print()

...yszczanie. Panel sterowania urządzenie posiada LED. Oświetlenie czytelne, ale nie oślepiające.
Czujnik ruchu. Co do jego wielkości - nie zauważyłam żadnej różnicy, a przy malutkiej podstawie
wygląda ok.



In [32]:
from tqdm import tqdm

set_seed(11)

augmented_dataset = []

for idx, row in tqdm(dataset.iterrows(), total=len(dataset)):
    text = row["text"]
    label = row["Finalna anotacja"]
    augmented_dataset.append({"text": text, "label": label, "is_generated": False})

    text_as_inputs = tokenizer(text, return_tensors="pt").to(device)

    generated = model.generate(
        **text_as_inputs,
        max_new_tokens=40,
        min_new_tokens=10,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=3,
    )

    generated_texts = decode_generated_text(text, generated, with_ellipsis=False)
    for generated_text in generated_texts:
        augmented_dataset.append(
            {"text": generated_text, "label": label, "is_generated": True}
        )

augmented_dataset = pd.DataFrame(augmented_dataset)

100%|██████████| 138/138 [00:28<00:00,  4.92it/s]


In [33]:
for line in augmented_dataset.iterrows():
    print(f"Label: {line[1]['label']}", line[1]["text"])

Label: 2 "Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda (chociaż wolałbym, gdyby był matowy, ale to inna kwestia), a także sztywno trzyma pada. Jedyny, drobny minus to zamek błyskawiczny, który przy otwarciu w dziwny sposób się ""rozkłada"", ale można to samemu sobie poprawić, poza tym nie wpływa to na nic."
Label: 2 "Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda (chociaż wolałbym, gdyby był matowy, ale to inna kwestia), a także sztywno trzyma pada. Jedyny, drobny minus to zamek błyskawiczny, który przy otwarciu w dziwny sposób się ""rozkłada"", ale można to samemu sobie poprawić, poza tym nie wpływa to na nic." - nie zauważyłem, żeby w innych Xboksach zamek nie działał.
Label: 2 "Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda (chociaż wolałbym, gdyby był matowy, ale to inna kwestia), a także sztywno trzyma

In [35]:
augmented_dataset.to_csv(
    "data/annotations_all_batches - SENTENCE_AUGMENTED_GPT2.csv", index=False
)